<a href="https://colab.research.google.com/github/luisosmx/web-scraperf1/blob/main/poc_webscrapper_f1data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


# GET request to URL:
url = "https://www.formula1.com/en/results.html/2023/team.html"
response = requests.get(url)

In [9]:
# Analizar el contenido HTML usando BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

In [10]:
# Encontrar la tabla de resultados para los conductores
table = soup.find("table", attrs={"class": "resultsarchive-table"})

# Extraer los datos de la tabla en un DataFrame de pandas
df = pd.read_html(str(table))[0]

In [11]:
print(df)

   Unnamed: 0  Pos                          Team  PTS  Unnamed: 4
0         NaN    1    Red Bull Racing Honda RBPT  123         NaN
1         NaN    2  Aston Martin Aramco Mercedes   65         NaN
2         NaN    3                      Mercedes   56         NaN
3         NaN    4                       Ferrari   26         NaN
4         NaN    5              McLaren Mercedes   12         NaN
5         NaN    6                Alpine Renault    8         NaN
6         NaN    7                  Haas Ferrari    7         NaN
7         NaN    8            Alfa Romeo Ferrari    6         NaN
8         NaN    9         AlphaTauri Honda RBPT    1         NaN
9         NaN   10             Williams Mercedes    1         NaN


In [12]:
# Seleccionar sólo las columnas necesarias
df = df[["Pos", "Team", "PTS"]]

# Crear una nueva columna "Acronym" con los últimos 3 caracteres de la columna "Driver"
#df["Acronym"] = df["Driver"].str.slice(-3)

# Quitar los últimos 3 caracteres de la columna "Driver"
#df["Driver"] = df["Driver"].str.slice(stop=-3)


now = datetime.now()

epoch_time = int(now.timestamp())
  
df["Date"] = epoch_time

new_name = {"Pos": "Position", "PTS": "Points"}
df = df.rename(columns=new_name)

# Convertir la columna "PTS" a un tipo de datos de cadena
df["Points"] = df["Points"].astype(str)
df["Date"] = df["Date"].astype(str)
df["Position"] = df["Position"].astype(str)

# Imprimir el DataFrame resultante
#print(df)

<ipython-input-12-4ff935e77989>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = epoch_time


In [13]:
print(df.dtypes)

Position    object
Team        object
Points      object
Date        object
dtype: object


In [14]:
# Descarga del df en archivo .CSV
df.to_csv("resultados_drivers.csv", index=False)

In [ ]:
!pip install google-cloud-bigquery


In [ ]:

from google.cloud import bigquery


In [ ]:
auth.authenticate_user()

In [ ]:
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "formula1-data-382219.data_F1.results_drivers"


job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("Position", bigquery.enums.SqlTypeNames.STRING)
        
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

RefreshError: ignored

In [ ]:
import pandas_gbq
pandas_gbq.to_gbq(df, 'data_F1.results_drivers', project_id='formula1-data-382219', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 6786.90it/s]


In [ ]:
!pip install google-cloud-bigquery
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import auth
import pandas_gbq
from google.cloud import bigquery
auth.authenticate_user()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# GET request to URL:
url = "https://www.formula1.com/en/results.html/2022/drivers.html"
response = requests.get(url)

In [ ]:
def data_extraction(url: str) -> pd.DataFrame:

  response = requests.get(url)

  # Analizar el contenido HTML usando BeautifulSoup
  soup = BeautifulSoup(response.content, "html.parser")
  # Encontrar la tabla de resultados para los conductores
  table = soup.find("table", attrs={"class": "resultsarchive-table"})

  # Extraer los datos de la tabla en un DataFrame de pandas
  df = pd.read_html(str(table))[0]
  return df

In [ ]:
def data_tranformation(df: pd.DataFrame) -> pd.DataFrame:
  # Seleccionar sólo las columnas necesarias
  df = df[["Pos", "Driver", "Nationality", "Car", "PTS"]]

  # Crear una nueva columna "Acronym" con los últimos 3 caracteres de la columna "Driver"
  df["Acronym"] = df["Driver"].str.slice(-3)

  # Quitar los últimos 3 caracteres de la columna "Driver"
  df["Driver"] = df["Driver"].str.slice(stop=-3)


  now = datetime.now()

  epoch_time = int(now.timestamp())
    
  df["Date"] = epoch_time

  new_name = {"Pos": "Position","Car": "Team", "PTS": "Points"}
  df = df.rename(columns=new_name)
  return df

In [ ]:
def load_data(df: pd.DataFrame) -> str:
  try:
    auth.authenticate_user()
    client = bigquery.Client(project='formula1-data-382219')
    pandas_gbq.to_gbq(df, 'data_F1.results_drivers', project_id='formula1-data-382219', if_exists='replace')
    return "OK"
  except Exception as e:
    print("########################")
    print(str(e))
    print("########################")
    return "Error"

In [ ]:
def main():
  raw_data = data_extraction(url)
  transformed_data = data_tranformation(raw_data)
  result = load_data(transformed_data)
  print(result)
  return result

In [ ]:
main()

<ipython-input-13-d3824c9acddc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Acronym"] = df["Driver"].str.slice(-3)
<ipython-input-13-d3824c9acddc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Driver"] = df["Driver"].str.slice(stop=-3)
100%|██████████| 1/1 [00:00<00:00, 1216.09it/s]

OK


'OK'

In [ ]:
functions-framework==3.*
beautifulsoup4
pandas
pandas-gbq
requests
google-cloud-bigquery
google-colab